In [ ]:
## mount drive for access to the files
from google.colab import drive

drive.mount("/content/drive")

## all the drive the files are present in "/content/drive/My Drive"
# !ls "/content/drive/My Drive/Beuth Uni/Master Thesis/jenga"

import sys
sys.path.append('/content/drive/My Drive/Beuth Uni/Master Thesis/jenga')

In [ ]:
!pip install openml
!pip install pyod

!pip install mxnet autogluon
!pip install mxnet-mkl --pre --upgrade

In [ ]:
import random
import numpy as np
import pandas as pd

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from jenga.basis import Dataset
from jenga.corruptions.generic import MissingValues, SwappedValues
from jenga.corruptions.numerical import Scaling, GaussianNoise
from jenga.cleaning.ppp import PipelinePerformancePrediction
from jenga.cleaning.outlier_detection import NoOutlierDetection, PyODKNNOutlierDetection, PyODIsolationForestOutlierDetection, AutoGluonOutlierDetection
from jenga.cleaning.imputation import NoImputation, MeanModeImputation, AutoGluonImputation
from jenga.cleaning.clean import Clean

In [ ]:
seed = 100

In [ ]:
def run_experiment(dataset_name, learner, param_grid, corruptions, fraction, cleaners, num_repetitions, categorical_precision_threshold=0.7, numerical_std_error_threshold=2.0):
    
    ## dataset
    dataset = Dataset(seed, dataset_name)
    
    all_data = dataset.all_data
    attribute_names = dataset.attribute_names
    attribute_types = dataset.attribute_types
    
    ## categorical and numerical features
    categorical_columns = dataset.categorical_columns
    numerical_columns = dataset.numerical_columns
    print(f"Found {len(categorical_columns)} categorical and {len(numerical_columns)} numeric features \n")
    
    ## train and test data
    df_train, lab_train, df_test, lab_test = dataset.get_train_test_data()
    
    
    ## pipeline performance prediction (ppp)
    ppp = PipelinePerformancePrediction(seed, df_train, lab_train, df_test, lab_test, categorical_columns, numerical_columns, learner, param_grid)
    ppp_model = ppp.fit_ppp(df_train)
    
    ## generate corrpted data
    df_corrupted, perturbations, cols_perturbed, summary_col_corrupt = ppp.get_corrupted(df_test, corruptions, fraction, num_repetitions)
    
    ## cleaning
    clean = Clean(df_train, df_corrupted, categorical_columns, numerical_columns, categorical_precision_threshold, numerical_std_error_threshold, ppp, ppp_model, cleaners)
    df_outliers, df_cleaned, corrupted_score_ppp, best_cleaning_score, cleaner_scores_ppp, summary_cleaners = clean(df_train, df_test, df_corrupted, cols_perturbed)
    
    ## results
    result = {
        'ppp_score_model': ppp.predict_score_ppp(ppp_model, df_test),
        'ppp_score_corrupted': corrupted_score_ppp,
        'ppp_score_cleaned': best_cleaning_score,
        'ppp_scores_cleaners': cleaner_scores_ppp
    }
#     print('\n'.join([f'{key}:{val}' for key, val in result.items()]))
    
    ## summary
    summary = {
        'dataset': dataset_name,
        'model': learner,
        'corruptions': summary_col_corrupt,
        'cleaners': summary_cleaners,
        'result': result
    }
#     print('\n\n\n\n'.join([f'{key}:{val}' for key, val in summary.items()]))
    
    return summary #summary_col_corrupt, result

### Missing Values

In [ ]:
corruptions = [GaussianNoise]

In [ ]:
fractions = [0.15, 0.25, 0.5, 0.75, 0.9]

In [ ]:
cleaners = [
    (NoOutlierDetection, MeanModeImputation),
    (PyODKNNOutlierDetection, MeanModeImputation),
    (PyODKNNOutlierDetection, AutoGluonImputation),
    (PyODIsolationForestOutlierDetection, MeanModeImputation),
    (PyODIsolationForestOutlierDetection, AutoGluonImputation),
    (AutoGluonOutlierDetection, AutoGluonImputation)
]

#### Parkinsons

In [ ]:
dataset = 'parkinsons'

##### Stochastic Gradient Descent

In [ ]:
## model parameters
learner = SGDClassifier(loss='log')
param_grid = {
    'learner__max_iter': [500, 1000, 5000],
    'learner__penalty': ['l2', 'l1', 'elasticnet'],
    'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
    }

In [ ]:
results = []

In [ ]:
stdoutOrigin=sys.stdout 
sys.stdout = open("/content/drive/My Drive/Beuth Uni/Master Thesis/jenga/out/GaussianNoise/parkinsons_sgd.txt", "w")

for _ in range(10):
  print("............................................. ITERATION .............................................")
  for fraction in fractions:
    results.append(run_experiment(dataset, learner, param_grid, corruptions, fraction, cleaners, 100))

sys.stdout.close()
sys.stdout=stdoutOrigin

##### Random Forest

In [ ]:
## model parameters
learner = RandomForestClassifier()
param_grid = {
    'learner__n_estimators': [100, 200, 500],
    'learner__max_depth': [5, 10, 15]
    }

In [ ]:
results = []

In [ ]:
stdoutOrigin=sys.stdout 
sys.stdout = open("/content/drive/My Drive/Beuth Uni/Master Thesis/jenga/out/GaussianNoise/parkinsons_rf.txt", "w")

for _ in range(10):
  print("............................................. ITERATION .............................................")
  for fraction in fractions:
    results.append(run_experiment(dataset, learner, param_grid, corruptions, fraction, cleaners, 100))

sys.stdout.close()
sys.stdout=stdoutOrigin

#### Cleve

In [ ]:
dataset = 'cleve'

##### Stochastic Gradient Descent

In [ ]:
## model parameters
learner = SGDClassifier(loss='log')
param_grid = {
    'learner__max_iter': [500, 1000, 5000],
    'learner__penalty': ['l2', 'l1', 'elasticnet'],
    'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
    }

In [ ]:
results = []

In [ ]:
stdoutOrigin=sys.stdout 
sys.stdout = open("/content/drive/My Drive/Beuth Uni/Master Thesis/jenga/out/GaussianNoise/cleve_sgd.txt", "w")

for _ in range(10):
  print("............................................. ITERATION .............................................")
  for fraction in fractions:
    results.append(run_experiment(dataset, learner, param_grid, corruptions, fraction, cleaners, 100))

sys.stdout.close()
sys.stdout=stdoutOrigin

##### Random Forest

In [ ]:
## model parameters
learner = RandomForestClassifier()
param_grid = {
    'learner__n_estimators': [100, 200, 500],
    'learner__max_depth': [5, 10, 15]
    }

In [ ]:
results = []

In [ ]:
stdoutOrigin=sys.stdout 
sys.stdout = open("/content/drive/My Drive/Beuth Uni/Master Thesis/jenga/out/GaussianNoise/cleve_rf.txt", "w")

for _ in range(10):
  print("............................................. ITERATION .............................................")
  for fraction in fractions:
    results.append(run_experiment(dataset, learner, param_grid, corruptions, fraction, cleaners, 100))

sys.stdout.close()
sys.stdout=stdoutOrigin

#### Credit-g

In [ ]:
dataset = 'credit-g'

##### Stochastic Gradient Descent

In [ ]:
## model parameters
learner = SGDClassifier(loss='log')
param_grid = {
    'learner__max_iter': [500, 1000, 5000],
    'learner__penalty': ['l2', 'l1', 'elasticnet'],
    'learner__alpha': [0.0001, 0.001, 0.01, 0.1]
    }

In [ ]:
results = []

In [ ]:
stdoutOrigin=sys.stdout 
sys.stdout = open("/content/drive/My Drive/Beuth Uni/Master Thesis/jenga/out/GaussianNoise/credit-g_sgd.txt", "w")

for _ in range(10):
  print("............................................. ITERATION .............................................")
  for fraction in fractions:
    results.append(run_experiment(dataset, learner, param_grid, corruptions, fraction, cleaners, 100))

sys.stdout.close()
sys.stdout=stdoutOrigin

##### Random Forest

In [ ]:
## model parameters
learner = RandomForestClassifier()
param_grid = {
    'learner__n_estimators': [100, 200, 500],
    'learner__max_depth': [5, 10, 15]
    }

In [ ]:
results = []

In [ ]:
stdoutOrigin=sys.stdout 
sys.stdout = open("/content/drive/My Drive/Beuth Uni/Master Thesis/jenga/out/GaussianNoise/credit-g_rf.txt", "w")

for _ in range(10):
  print("............................................. ITERATION .............................................")
  for fraction in fractions:
    results.append(run_experiment(dataset, learner, param_grid, corruptions, fraction, cleaners, 100))

sys.stdout.close()
sys.stdout=stdoutOrigin